In [2]:
print("hello")

hello


In [3]:
import os 

In [4]:
%pwd

'd:\\WorkANDProjects\\Kidney-Disease-Classification-Deep-Learning-Project\\research'

In [5]:
os.chdir("d:/WorkANDProjects/Kidney-Disease-Classification-Deep-Learning-Project")

In [6]:
%pwd

'd:\\WorkANDProjects\\Kidney-Disease-Classification-Deep-Learning-Project'

In [14]:
import os
import shutil
from pathlib import Path

# Original dataset path
original_dataset = r"D:\WorkANDProjects\Kidney-Disease-Classification-Deep-Learning-Project\artifacts\data_ingestion\kidney-ct-scan-image"

# New subset dataset path
subset_dataset = r"D:\WorkANDProjects\Kidney-Disease-Classification-Deep-Learning-Project\artifacts\data_ingestion\subset_kidney_ct_scan"

# Number of images per class
MAX_IMAGES = 500  

def create_subset(original_dataset, subset_dataset, max_images=500):
    os.makedirs(subset_dataset, exist_ok=True)

    for class_name in os.listdir(original_dataset):
        class_path = os.path.join(original_dataset, class_name)
        subset_class_path = os.path.join(subset_dataset, class_name)

        if os.path.isdir(class_path):
            os.makedirs(subset_class_path, exist_ok=True)

            images = os.listdir(class_path)[:max_images]  # Take first N images
            for img in images:
                src = os.path.join(class_path, img)
                dst = os.path.join(subset_class_path, img)
                shutil.copy(src, dst)

    print(f"✅ Subset created at: {subset_dataset}")

if __name__ == "__main__":
    create_subset(original_dataset, subset_dataset, MAX_IMAGES)


✅ Subset created at: D:\WorkANDProjects\Kidney-Disease-Classification-Deep-Learning-Project\artifacts\data_ingestion\subset_kidney_ct_scan


In [22]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [23]:

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [24]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "subset_kidney_ct_scan")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [25]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [26]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [27]:
import os

print(os.listdir("artifacts/data_ingestion"))

['data.zip', 'kidney-ct-scan-image', 'subset_kidney_ct_scan']


In [28]:

try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-08-30 17:54:24,434: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-30 17:54:24,439: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-30 17:54:24,439: INFO: common: created directory at: artifacts]
[2025-08-30 17:54:24,439: INFO: common: created directory at: artifacts\training]
Found 200 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Epoch 1/10
12/12 [==============================] - 96s 8s/step - loss: 0.7511 - accuracy: 0.4986 - val_loss: 0.5843 - val_accuracy: 0.7448
Epoch 2/10
12/12 [==============================] - 94s 8s/step - loss: 0.6975 - accuracy: 0.5448 - val_loss: 0.5781 - val_accuracy: 0.7448
Epoch 3/10
12/12 [==============================] - 92s 8s/step - loss: 0.6968 - accuracy: 0.5312 - val_loss: 0.5725 - val_accuracy: 0.7500
Epoch 4/10
12/12 [==============================] - 91s 8s/step - loss: 0.7109 - accuracy: 0.4959 - val_loss: 0.5698 - val_accuracy: 0.7552
Epoch 5/10
12/12 [==========

In [29]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available: 0


In [1]:
import torch
print(torch.cuda.is_available())   # should be True
print(torch.cuda.get_device_name(0))  # should show Quadro T1000


True
Quadro T1000
